Import The Library

In [1]:
import pygame as pg
import numpy as np
import math
import numba

pygame 2.6.0 (SDL 2.28.4, Python 3.12.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
res = width, height = 550, 400 # Equlivant to res = (width, height)
offset = np.array([1.3 * width, height]) // 2 # Multiplies the width by 1.3 sets the height the same, then dives the result array by half
maxItteration = 150
zoom = 2.2 / height

texture = pg.image.load('texture.jpg') # Load the texture file
textureSize = min(texture.get_size()) - 1 # Get the size of the texture
textureArray = pg.surfarray.array3d(texture) # Make a pygame pixel array of the texture

class mandlebrotFractal:
    def __init__(self, app):
        self.app = app;
        self.screenArray = np.full((width, height, 3), [0,0,0], dtype = np.uint8)
        '''
        The np.full command creates an array in this case with 3 deminsions, where the deminsions
        are given by the width, height, and 3. And initialzies the array to 0 at first with the
        datatype of unsigned int 8 bits which has a max of 255 which perfectly matches to RGB.
        '''
    @staticmethod
    @numba.njit(fastmath = True)
    def render(screenArray):
        for x in numba.prange(width):
            for y in numba.prange(height):
                c = (x - offset[0]) * zoom + 1j * (y - offset[1]) * zoom
                z = 0
                numberOfItteration = 0

                for i in numba.prange(maxItteration):
                    z = z ** 2 + c

                    if (z.real ** 2 + z.imag ** 2 > 8):
                        break
                    
                    numberOfItteration += 1
                
                colums = int(textureSize * numberOfItteration / maxItteration)
                screenArray[x, y] = textureArray[colums, colums]

        return screenArray
    
    def update(self):
        self.screenArray = self.render(self.screenArray)

    def draw(self):
        pg.surfarray.blit_array(self.app.screen, self.screenArray)

        '''
        The function pg.surfarray.blit_array in Pygame is used to copy pixel data from a NumPy array
        onto a pygame.Surface. Essentially, it allows you to update a Surface with pixel values directly
        from an array. Here’s a detailed explanation of what it does:
        '''
    
    def run(self):
        self.update()
        self.draw()

class mandlebrotApp: # Creates the GUI game loop
    def __init__(self):
        self.screen = pg.display.set_mode(res, pg.SCALED) # Sets the display of the window
        self.clock = pg.time.Clock() # pg.time.Clock() creates a Clock object, which you can use to regulate how fast the game loop runs
        self.mandlebrot = mandlebrotFractal(self)
    
    def run(self): # Self explanitory code, this is the game loop
        running = True
        
        while (running):
            for events in pg.event.get():
                if (events.type == pg.QUIT): # If the 'x' button is pressed the application closes
                    running = False

                if (events.type == pg.KEYDOWN): # If the letter 'q' is pressed the application closes
                    if (events.key == pg.K_q):
                        running = False

            self.screen.fill('black') # Makes the background balck
            self.clock.tick()
            self.mandlebrot.run()
            pg.display.flip() # Updates the window

            pg.display.set_caption(f'FPS: {self.clock.get_fps()}')  # Set the window caption to display the FPS

        pg.quit() # Closes the pygame client

def main():
    app = mandlebrotApp()
    app.run()

if __name__ == '__main__':
    main()